In [1]:
# changing core directory
import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)
os.chdir('..')

In [2]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%load_ext autoreload
%autoreload

In [3]:
path = 'checkpoints/movielens_1m'
all_files = [file for file in os.listdir(path)  if ".csv" in file ]
all_files

['model_sasrec_0|agg=exp_learnable_hawkes.csv',
 'model_sasrec_0.csv',
 'model_sasrec_0|agg=kernel_attention.csv',
 'model_sasrec_0|agg=max.csv',
 'model_sasrec_0|agg=attention.csv',
 'model_sasrec_0|agg=mean.csv',
 'model_sasrec_0|agg=learnable_attention.csv',
 'model_sasrec_0|agg=exp_hawkes.csv',
 'model_sasrec_0|agg=symmetrical_attention.csv']

In [9]:
all_res = pd.DataFrame([])

for file in all_files:
    df = pd.read_csv(os.path.join(path, file))
    df['dataset'] = 'movielens_1m'
    if len(file.split('|')) > 1:
        hp = file.split('|')[1]
        method = hp
        method = method.replace('.csv', '').replace('agg=', '')
        df['head'] = 'replace'
        df['agg_method'] = method
    else:
        df['head'] = '-'
        df['agg_method'] = 'no_agg'

    all_res = pd.concat([all_res, df])
    

all_res 

,Mode,Epoch,Train_Loss,Val_Loss,Val_NDCG,dataset,head,agg_method
0,external,0,2.133274e+32,5.853745e+33,0.014367,movielens_1m,replace,exp_learnable_hawkes
1,external,1,2.138385e+32,5.520207e+33,0.021319,movielens_1m,replace,exp_learnable_hawkes
2,external,2,2.162771e+32,5.308473e+33,0.026512,movielens_1m,replace,exp_learnable_hawkes
3,external,3,2.170690e+32,5.287630e+33,0.028058,movielens_1m,replace,exp_learnable_hawkes
4,external,4,2.143634e+32,5.285985e+33,0.029965,movielens_1m,replace,exp_learnable_hawkes
...,...,...,...,...,...,...,...,...
45,external,45,2.443200e+00,6.753100e+00,0.062938,movielens_1m,replace,symmetrical_attention
46,external,46,2.445400e+00,6.755600e+00,0.063468,movielens_1m,replace,symmetrical_attention
47,external,47,2.445900e+00,6.751300e+00,0.061841,movielens_1m,replace,symmetrical_attention
48,external,48,2.444300e+00,6.775500e+00,0.061479,movielens_1m,replace,symmetrical_attention


In [11]:
all_res.groupby("agg_method", as_index=False).agg({'Val_NDCG': 'max'}).sort_values('Val_NDCG')

,agg_method,Val_NDCG
2,exp_learnable_hawkes,0.043606
1,exp_hawkes,0.046789
3,kernel_attention,0.066620
7,no_agg,0.067446
8,symmetrical_attention,0.067895
5,max,0.068515
6,mean,0.068523
4,learnable_attention,0.068705
0,attention,0.069147


# renaming

In [6]:
# renaming

path = 'checkpoints/movielens_1m'
all_files = [file for file in os.listdir(path)  if "sarsrec" in file ]
all_files

new_name_files = []
for file in all_files:
    new_file = file
    new_file = new_file.replace('model=sarsrec_seed=0', 'new_head|model=sasrec_seed=0')
    new_name_files.append(new_file)


for old_file, new_file in zip(all_files, new_name_files):
    os.rename(os.path.join(path, old_file), os.path.join(path, new_file))